

    ~copy the initial database, DON"T MODIFY DIRECTLY.
    *Delete videos that are not available.


    *Process videos by cropping them based on the timestamps and rename in the the following format
        ~Label-videoID-1
        if there are more than 1 videos, 
        Label-videoID-2, 3, etc;


    *Crop the width and the height of the videos based of the box coordinates provided in the dataset.





In [1]:
import cv2
import numpy as np
from numpy import asarray
import pandas as pd
from PIL import Image
import os
import math

Here is the format of the videos  I am treating
{"org_text": "emotional", "clean_text": "emotional", "start_time": 0.0, "signer_id": 20, "signer": 40, "start": 0, "end": 30, "file": "SignSchool Emotional", "label": 907, "height": 360.0, "fps": 23.976, "end_time": 1.251, "url": "www.youtube.com/watch?v=C59jcSo4fEI", "text": "emotional", "box": [0.059554219245910645, 0.2810196578502655, 1.0, 0.8543376922607422], "width": 640.0},

{<given-word>, <start time> <>}

importing the data


In [2]:
# test_df = pd.read_json("Datasets/MS-ASL/MS-ASL-with-new-labels/NEW-MSASL-train-file.json")

changing videos to numpy array (function):

https://github.com/dxli94/WLASL/blob/master/code/I3D/datasets/nslt_dataset.py

In [3]:
# changing an image into an array
def change_img(image_name):
    img = Image.open(f"{image_name}")
 
    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(img)
    
    # <class 'numpy.ndarray'>
    print(type(numpydata))
    
    #  shape
    print(numpydata.shape)


# change_img("test_frames/frame_330.jpg")

cutting the videos


splitting the videos into frames.

https://techtutorialsx.com/2021/04/29/python-opencv-splitting-video-frames/

In [4]:
# def load_rgb_frames_from_video(vid_root, vid, start, num, resize=(256, 256)):
#     video_path = os.path.join(vid_root, vid + '.mp4')

#     vidcap = cv2.VideoCapture(video_path)

#     frames = []

#     total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

#     vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
#     for offset in range(min(num, int(total_frames - start))):
#         success, img = vidcap.read()

#         w, h, c = img.shape
#         if w < 226 or h < 226:
#             d = 226. - min(w, h)
#             sc = 1 + d / min(w, h)
#             img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

#         if w > 256 or h > 256:
#             img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

#         img = (img / 255.) * 2 - 1

#         frames.append(img)

#     return np.asarray(frames, dtype=np.float32)

# #num is the number of frames
# li = load_rgb_frames_from_video("IMAGE_vision", "test-video.mp4", 0, 60)
# for i in li:
#     print(i)
#     print("hi")
#     type(li)

In [5]:
def get_image(video_url, video_path, start_time, end_time, fps):
    capture = cv2.VideoCapture(f"Datasets/validation_video_data/validation_video_data/{video_url}.mp4")


    frameNr = 0
    # fps=int(capture.get(cv2.CAP_PROP_FRAME_COUNTCAP))

    #time in seconds
    # start_time=start_time
    # end_time=end_time

    #size of the image
    # frameWidth = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frameHeight = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # frameWidth = width
    # frameHeight = height
    
    # frameCount = (end_time-start_time)

    start_frame=start_time*fps
    end_frame=end_time*fps

    # buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

    frames=[]
    while (True):
        success, img = capture.read()
        # print(frameNr)
        if success and frameNr>start_frame and frameNr<end_frame:
            if frameNr%30==0: 
                # #transform into numpy array
                w, h, c = img.shape

                if w < 226 or h < 226:
                    d = 226. - min(w, h)
                    sc = 1 + d / min(w, h)
                    img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

                if w > 256 or h > 256:
                    img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

                img = (img / 255.) * 2 - 1

                frames.append(img)

                #this line used to write it into a folder
                # cv2.imwrite(f'{video_url}/frame_{frameNr}.jpg', img)

        #end of the video, or end of the portion of the video
        if frameNr>end_frame:
            break
    
        frameNr = frameNr+1

    capture.release()


    #return a numpy array.
        #saves the video in label/url.npy
    mode=0o666

    
    try:
        #if file doesn't exist, create it   
        if os.path.isdir(f"val_data/{video_path}") == False:
            os.mkdir(f"val_data/{video_path}", mode)
        np.save(f"val_data/{video_path}/{video_url}", np.asarray(frames, dtype=np.float32), allow_pickle=True, fix_imports=True)
    
    except FileNotFoundError:
        pass

    return np.asarray(frames, dtype=np.float32)
        



#just give me the video path in here
# get_image(" ALY PUT THE VIDEO PATH, RUN SOME LOOP OR SOMETHING, and it should be fine.", "VIDEO URL (which will be the name of the folder)", "start_time", "endtime")

# get_image("alkfjdsl2", "test-video2", 5, 30, 29.97)

creating the folders for the labels


In [6]:
test_df = pd.read_json('Datasets/MS-ASL/MS-ASL-with-new-labels/NEW-MSASL_val.json')
test_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,available
0,absent,absent,0.0,114,-1,0,37,ASL ABSENT,837,360,28.971,1.277,https://www.youtube.com/watch?v=ri3NrdgfAtE,absent,"[0.21896389130000002, 0.008568197500000001, 0....",202,NaN,False
1,help,help,0.0,76,42,0,110,help 2,50,360,29.970,3.670,www.youtube.com/watch?v=l31UXgChCS4,help,"[0.0503727198, 0.2994125783, 1, 0.6968145967]",640,NaN,True
2,come on,come on,0.0,114,-1,0,41,asl come on,889,360,25.000,1.640,https://www.youtube.com/watch?v=pt9bV_EvcaU,come on,"[0.08946925400000001, 0.1794851124, 0.99899017...",480,NaN,False
3,LANGUAGE,language,0.0,3,-1,0,56,LANGUAGE(3),513,360,15.000,3.733,https://www.youtube.com/watch?v=-j1wozf6o9w,language,"[0.1770857871, 0.0036684573, 1, 0.9955700636]",480,NaN,True
4,confused,confused,0.0,53,-1,0,95,ASL Confused,272,360,29.969,3.170,https://www.youtube.com/watch?v=y8tHmOQcCwU,confused,"[0.0626253188, 0.209987849, 1, 0.7668771744]",640,NaN,False


In [7]:
hi="hello/hi"
str(hi)
hi=hi.replace("/", " ")
print(hi)
hi=hi[:-1]
print(hi)

hello hi
hello h


In [8]:
for idx in range(0, len(test_df)-1):
    #if the file is available:
    if test_df.loc[idx, "available"]==False:
        continue


    #file-name
    file_name=test_df.loc[idx, "url"][-11:]#[24:]
    # org_text=org_text.replace("/", " ")


    start_time = test_df.loc[idx, "start_time"]

    end_time=test_df.loc[idx, "end_time"]

    fps=test_df.loc[idx, "fps"]

    clean_text=test_df.loc[idx, "clean_text"]
    #replacing all odd characters
    str(clean_text)
    clean_text=clean_text.replace("/", " ")
    if clean_text[-1]==" ":
        clean_text[:-1]

    #if your, I am at "start" 2578, /25 for the idxth word
    if idx%10==0:
        print(idx)

    #if file was already created:
    if os.path.exists(f"val_data/{clean_text}/{file_name}.npy") == True:
        continue

    get_image(file_name, clean_text, start_time, end_time, fps)
    

    

50
60
70
80
90
100
170
180
190
230
240
250
260
270
280
290
300
310
320
330
350
370
410
420
430
440
450
460
470
480
490
500
510
520
530
580
590
600
620
640
660
680
730
740
750
760
770
780
790
800
810
840
850
860
880
900
910
920
940
950
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1330
1340
1350
1360
1370
1450
1460
1470
1490
1500
1510
1520
1530
1540
1550
1560
1580
1590
1600
1610
1630
1640
1650
1860
1870
